In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import pandas as pd

from dynamicslib.consts import muEM
from dynamicslib.common import manifold_stepoffs, prop_multiple, jacobi_constant, coupled_stm_eom, eom
from dynamicslib.integrator import dop853
from dynamicslib.common_targetters import *
from dynamicslib.targeter import dc_square, dc_underconstrained, dc_overconstrained
from numba import njit

from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fam1 = "L1 Lyapunov"
fam2 = "L2 Lyapunov"
JC_target = 3.1

root = os.getcwd()
df1 = pd.read_csv(f"{root}/database/{fam1}.csv").set_index("Index")
df2 = pd.read_csv(f"{root}/database/{fam2}.csv").set_index("Index")
if "Initial z" not in df1.columns:
    df1["Initial z"] = 0
if "Initial z" not in df2.columns:
    df2["Initial z"] = 0
ind1 = np.argmin(np.abs(df1["Jacobi Constant"]-JC_target))
ind2 = np.argmin(np.abs(df2["Jacobi Constant"]-JC_target))
stab1 = df1.iloc[ind1]["Stability Index"]
stab2 = df2.iloc[ind2]["Stability Index"]
print(stab1, stab2)

In [ ]:
int_tol = 1e-12
targetter = JC_fixed_spatial_perpendicular(int_tol, JC_target, muEM)

In [ ]:
X1 = df1[["Initial x", "Initial z", "Initial vy", "Period"]].values[ind1]
X2 = df2[["Initial x", "Initial z", "Initial vy", "Period"]].values[ind2]
X1[-1] /= 2
X2[-1] /= 2

x02 = targetter.get_x0(X2)
x01 = targetter.get_x0(X1)

In [ ]:
X1, _, _ = dc_square(X1, targetter.f_df_stm, 1e-13, debug=True)
x01, tf1 = targetter.get_x0(X1), targetter.get_tf(X1)
X2, _, _ = dc_square(X2, targetter.f_df_stm, 1e-13, debug=True)
X2[1] *= -1
x02, tf2 = targetter.get_x0(X2), targetter.get_tf(X2)

In [ ]:
@njit(cache=True)
def moon(t, x, mu):
    return x[0] - (1 - mu)


@njit(cache=True)
def event2(t, x, mu):
    return x[2]


@njit(cache=True)
def exit_right(t, x, mu):
    return x[0] - 1.6


@njit(cache=True)
def exit_left(t, x, mu):
    return x[0] - 0.6


# @njit(cache=True)
# def exit_sides(t, x, mu):
#     return np.abs(x[1]) - 1


n_event = 0

event2.terminal = n_event
event2.direction = 0
moon.terminal = n_event
moon.direction = 0
exit_right.terminal = 1
exit_left.terminal = 1
exit_right.direction = 1
exit_left.direction = -1
# exit_sides.terminal = 1

events = [exit_right, exit_left, moon]

In [ ]:
N_manifolds = 150
step = 1e-3
x0s_u, _, aux_u = manifold_stepoffs(x01, tf1, N_manifolds, step, muEM, 1e-12)
_, x0s_s, aux_s = manifold_stepoffs(x02, tf2, N_manifolds, step, muEM, 1e-12)

In [ ]:
out_u = solve_ivp(eom, (0.0, tf1), x01, "DOP853", atol=1e-12, rtol=1e-12, args=(muEM,))
out_s = solve_ivp(eom, (0.0, -tf2), x02, "DOP853", atol=1e-12, rtol=1e-12, args=(muEM,))

In [ ]:
manifolds_u = prop_multiple(np.array(x0s_u), events, 30.0, muEM, 1e-11)
print("done w 1")
manifolds_s = prop_multiple(np.array(x0s_s), events, -30.0, muEM, 1e-11)
print("done w 2")

In [ ]:
ye_u = {
    (k, jj): v.y_events[-1][jj]
    for k, v in manifolds_u.items()
    for jj in range(len(v.y_events[-1]))
    if len(v.y_events[-1])
}
ye_s = {
    (k, jj): v.y_events[-1][jj]
    for k, v in manifolds_s.items()
    for jj in range(len(v.y_events[-1]))
    if len(v.y_events[-1])
}
ye_u = {k: np.array([*v[:3], *v[3:]]) for k, v in ye_u.items()}
ye_s = {k: np.array([*v[:3], *v[3:]]) for k, v in ye_s.items()}

In [ ]:
rtol = 5e-2
vtol = 5e-2

dists = [
    ((k1, k2), np.linalg.norm(v1 - v2))
    for k2, v2 in ye_s.items()
    for k1, v1 in ye_u.items()
    if np.linalg.norm((v1 - v2)[:3]) <= rtol and np.linalg.norm((v1 - v2)[3:]) <= vtol
]
len(dists)

In [ ]:
ind_use = 0  # 50

best_set = dists[np.argsort([d[1] for d in dists])[ind_use]]
(i1, ie1), (i2, ie2) = best_set[0]
te1 = manifolds_u[i1].t_events[-1][ie1]
te2 = manifolds_s[i2].t_events[-1][ie2]
it1 = list(np.abs(manifolds_u[i1].t) >= np.abs(te1)).index(True)
it2 = list(np.abs(manifolds_s[i2].t) >= np.abs(te2)).index(True)

x0_u, phi_u, e_u, _ = [var[i1 % N_manifolds].copy() for var in aux_u]
x0_s, phi_s, _, e_s = [var[i2 % N_manifolds].copy() for var in aux_s]
if i1 >= N_manifolds:
    e_u *= -1
if i2 >= N_manifolds:
    e_s *= -1

scale = 1.
targetter2 = manifold_reduced_dim(1e-10, x0_u, x0_s, e_u, e_s, scale, muEM)
# targetter2 = heterclinic(1e-10, x0_u, x0_s, phi_u, phi_s, e_u, e_s, scale, muEM)
# targetter2 = heterclinic_mod(1e-10, 0.0, x0_u, x0_s, phi_u, phi_s, e_u, e_s, 1e-3, muEM)

In [ ]:
X = np.array([0, te1, 0, te2])
# X = np.array([step * scale, te1, step * scale, te2])
f, df, _ = targetter2.f_df_stm(X)
Xnew, _, _ = dc_underconstrained(
    X, targetter2.f_df_stm, 1e-8, max_step=1e-2, fudge=0.5, debug=True, max_iter=2000
)

In [ ]:
man_u = solve_ivp(
    eom,
    (0.0, targetter2.get_tf_u(Xnew)),
    targetter2.get_x0_unstable(Xnew),
    "DOP853",
    atol=1e-10,
    rtol=1e-10,
    args=(muEM,),
)
man_s = solve_ivp(
    eom,
    (0.0, targetter2.get_tf_s(Xnew)),
    targetter2.get_x0_stable(Xnew),
    "DOP853",
    atol=1e-10,
    rtol=1e-10,
    args=(muEM,),
)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(projection="3d")
for dictionary, color in zip([manifolds_u, manifolds_s], ["r", "b"]):
    for ind in dictionary.keys():
        out = dictionary[ind]
        ax.plot(*out.y[:3], f"-{color}", alpha=0.025)
ax.plot(*out_u.y[:3], "-k", lw=3)
ax.plot(*out_s.y[:3], "-k", lw=3)
ax.plot(*man_u.y[:3], f"-k", lw=1)
ax.plot(*man_s.y[:3], f"-k", lw=1)
ax.set(
    xlim=(0.7, 1.3),
    ylim=(-0.2, 0.2),
    zlim=(-0.2, 0.2),
    xlabel="x [LU]",
    ylabel="y [LU]",
    aspect="equal",
)
# fig.colorbar()
# ax.axis("equal")
plt.grid(True)
plt.show()

In [ ]:
%matplotlib ipympl
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection="3d")

ax.plot(*out_u.y[:3], "-k", lw=3)
ax.plot(*out_s.y[:3], "-k", lw=3)
cm = plt.get_cmap("rainbow")
jmax = len(dists)
iterator = list(range(jmax))

# iterator = list(range(12,17))

sc = ax.scatter(*[[np.nan]*len(iterator)]*3, c=iterator,cmap=cm,vmin=min(iterator),vmax=max(iterator))
for jj in iterator:
    best_set = dists[np.argsort([d[1] for d in dists])[jj]]
    (i1, ie1), (i2, ie2) = best_set[0]
    te1 = manifolds_u[i1].t_events[-1][ie1]
    te2 = manifolds_s[i2].t_events[-1][ie2]
    it1 = list(np.abs(manifolds_u[i1].t) >= np.abs(te1)).index(True)
    it2 = list(np.abs(manifolds_s[i2].t) >= np.abs(te2)).index(True)

    cv = (jj-min(iterator))/(max(iterator)-min(iterator))
    ax.plot(*manifolds_u[i1].y[:3, : it1 + 1], c=cm(cv), lw=1)
    ax.plot(*manifolds_s[i2].y[:3, : it2 + 1], c=cm(cv), lw=1)
    # ax.scatter(*manifolds_u[i1].y_events[-1][ie1][:3], color=cm(jj / jmax))
    # ax.scatter(*manifolds_s[i2].y_events[-1][ie2][:3], color=cm(jj / jmax))

    # ax.text(*manifolds_s[i2].y[:2,20], str(jj), color=cm(jj / jmax))
ax.set(
    # xlim=(0.75, 1.25),
    # ylim=(-0.2, 0.2),
    # zlim=(-0.2, 0.2),
    # xlabel="x [LU]",
    # ylabel="y [LU]",
    aspect="equal",
)
div = make_axes_locatable(ax)
# cax = div.append_axes("right", size="2%", pad=0.1)
# fig.colorbar(sc, cax)
fig.colorbar(sc,ax=ax)
ax.grid(True)
plt.show()